In [ ]:
pip install colorama

Mounted at /content/drive


In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
df = pd.read_csv('imdb_top_1000.csv')
df['combined_features'] = df['Genre'].fillna('') + ' ' + df['Overview'].fillna('')
df.head()

Saving imdb_top_1000.csv to imdb_top_1000 (1).csv


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Number_of_Movies,combined_features
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",1,Drama Two imprisoned men bond over a number of...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",1,"Crime, Drama An organized crime dynasty's agin..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",1,"Action, Crime, Drama When the menace known as ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",1,"Crime, Drama The early life and career of Vito..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",1,"Crime, Drama A jury holdout attempts to preven..."


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from colorama import init, Fore
import time
import sys


# Initialize colorama
init(autoreset=True)

# Load and preprocess the dataset
# def load_data(file_path='/content/drive/MyDrive/Colab Notebooks/Codingal/AI Expert - Hands on AI,LLMs & Python/Mod1: Intro to AI/imdb_top_1000.csv'):
#     try:
#         df = pd.read_csv(file_path)
#         df['combined_features'] = df['Genre'].fillna('') + ' ' + df['Overview'].fillna('')
#         return df
#     except FileNotFoundError:
#         print(Fore.RED + f"Error: The file '{file_path}' was not found.")
#         exit()

# movies_df = load_data()




movies_df = df

# Vectorize the combined features and compute cosine similarity
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:
# List all unique genres
def list_genres(df):
    return sorted(set(genre.strip() for sublist in df['Genre'].dropna().str.split(', ') for genre in sublist))

genres = list_genres(movies_df)

# Recommend movies based on filters (genre, mood, rating)
def recommend_movies(genre=None, mood=None, rating=None, top_n=5):
    filtered_df = movies_df
    if genre:
        filtered_df = filtered_df[filtered_df['Genre'].str.contains(genre, case=False, na=False)]
    if rating:
        filtered_df = filtered_df[filtered_df['IMDB_Rating'] >= rating]

    filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)  # Randomize the order

    recommendations = []
    for idx, row in filtered_df.iterrows():
        overview = row['Overview']
        if pd.isna(overview):
            continue
        polarity = TextBlob(overview).sentiment.polarity
        if (mood and ((TextBlob(mood).sentiment.polarity < 0 and polarity > 0) or polarity >= 0)) or not mood:
            recommendations.append((row['Series_Title'], polarity))
        if len(recommendations) == top_n:
            break

    return recommendations if recommendations else "No suitable movie recommendations found."


In [ ]:
# Display recommendations
def display_recommendations(recs, name):
    print(Fore.YELLOW + f"\n🎬 AI-Analyzed Movie Recommendations for {name}:")
    for idx, (title, polarity) in enumerate(recs, 1):
        sentiment = "Positive 😊" if polarity < 0 else "Negative 😔" if polarity < 0 else "Neutral 😐"
        print(f"{Fore.CYAN}{idx}. 🎥 {title} (Polarity: {polarity:.2f}, {sentiment})")

# Small processing animation
def processing_animation():
    for _ in range(3):
        print(Fore.YELLOW + ".", end="", flush=True)
        time.sleep(0.5)

# Handle AI recommendation flow
def handle_ai(name):
    print(Fore.BLUE + f"\n🤖 Let's find the perfect movie for you!\n")

    # Show genres in a single line
    print(Fore.GREEN + "Available Genres: ", end="")
    for idx, genre in enumerate(genres, 1):
        print(f"{Fore.CYAN}{idx}. {genre}")
    print()  # To move to the next line after all genres are listed

    while True:
        genre_input = input(Fore.YELLOW + "Enter genre number or name: ").strip()
        if genre_input.isdigit() and 1 <= int(genre_input) <= len(genres):
            genre = genres[int(genre_input)-1]
            break
        elif genre_input.title() in genres:
            genre = genre_input.title()
            break
        print(Fore.RED + "Invalid input. Try again.\n")

    mood = input(Fore.YELLOW + "How do you feel today? (Describe your mood): ").strip()

        # Processing animation while analyzing mood
    print(Fore.BLUE + "\nAnalyzing mood", end="", flush=True)
    processing_animation()  # Small processing animation during mood analysis
    polarity = TextBlob(mood).sentiment.polarity
    mood_desc = "positive 😊" if polarity > 0 else "negative 😔" if polarity < 0 else "neutral 😐"
    print(f"\n{Fore.GREEN}Your mood is {mood_desc} (Polarity: {polarity:.2f}).\n")

    while True:
        rating_input = input(Fore.YELLOW + "Enter minimum IMDB rating (7.6-9.3) or 'skip': ").strip()
        if rating_input.lower() == 'skip':
            rating = None
            break
        try:
            rating = float(rating_input)
            if 7.6 <= rating <= 9.3:
                break
            print(Fore.RED + "Rating out of range. Try again.\n")
        except ValueError:
            print(Fore.RED + "Invalid input. Try again.\n")

    # Processing animation while finding movies
    print(f"{Fore.BLUE}\nFinding movies for {name}", end="", flush=True)
    processing_animation()  # Small processing animation while finding movies

    recs = recommend_movies(genre=genre, mood=mood, rating=rating, top_n=5)
    if isinstance(recs, str):
        print(Fore.RED + recs + "\n")
    else:
        display_recommendations(recs, name)


    while True:
      action = input(Fore.YELLOW + "\nWould you like more recommendations? (yes/no): ").strip().lower()
      if action == 'no':
          print(Fore.GREEN + f"\nEnjoy your movie picks, {name}! 🎬 🍿\n")
          break
      elif action == 'yes':
          recs = recommend_movies(genre=genre, mood=mood, rating=rating, top_n=5)
          if isinstance(recs, str):
              print(Fore.RED + recs + "\n")
          else:
              display_recommendations(recs, name)
      else:
          print(Fore.RED + "Invalid choice. Try again.\n")

# Main program
def main():
    print(Fore.BLUE + "🎬 Welcome to your Personal Movie Recommendation Assistant! 🎬\n")
    name = input(Fore.YELLOW + "What's your name? ").strip()

    print(f"\n{Fore.GREEN}Great to meet you, {name}!\n")
    handle_ai(name)

if __name__ == "__main__":
    main()




🎬 Welcome to your Personal Movie Recommendation Assistant! 🎬

What's your name? Mr.Agent

Great to meet you, Mr.Agent!


🤖 Let's find the perfect movie for you!

Available Genres: 1. Action
2. Adventure
3. Animation
4. Biography
5. Comedy
6. Crime
7. Drama
8. Family
9. Fantasy
10. Film-Noir
11. History
12. Horror
13. Music
14. Musical
15. Mystery
16. Romance
17. Sci-Fi
18. Sport
19. Thriller
20. War
21. Western

Enter genre number or name: History
How do you feel today? (Describe your mood): Good

Analyzing mood...
Your mood is positive 😊 (Polarity: 0.70).

Enter minimum IMDB rating (7.6-9.3) or 'skip': skip

Finding movies for Mr.Agent...
🎬 AI-Analyzed Movie Recommendations for Mr.Agent:
1. 🎥 Ayla: The Daughter of War (Polarity: 0.08, Neutral 😐)
2. 🎥 Joyeux Noël (Polarity: 0.04, Neutral 😐)
3. 🎥 All the President's Men (Polarity: 0.00, Neutral 😐)
4. 🎥 The Message (Polarity: 0.07, Neutral 😐)
5. 🎥 Andrei Rublev (Polarity: 0.00, Neutral 😐)

Would you like more recommendations? (yes/no): n

✅ Your EXISTING Colab notebook pushed!


In [21]:
!ls -la /content/
!pwd

total 884
drwxr-xr-x 1 root root   4096 Dec 14 21:36  .
drwxr-xr-x 1 root root   4096 Dec 14 20:08  ..
drwxr-xr-x 4 root root   4096 Dec 11 14:34  .config
drwxr-xr-x 8 root root   4096 Dec 14 21:36  .git
-rw-r--r-- 1 root root 440540 Dec 14 20:41 'imdb_top_1000 (1).csv'
-rw-r--r-- 1 root root 440540 Dec 14 20:38  imdb_top_1000.csv
drwxr-xr-x 1 root root   4096 Dec 11 14:34  sample_data
/content
